# 本文将用于从Tensorflow/TFLite中提取Weights

## For Dense-Mnist-Int8模型

## 1.从Tensorflow(.h5 File)中提取Conv2D的Weights

In [1]:
import numpy as np

#以下为一个提取Tensorflow Conv2d卷积weights+bias的函数，会以txt保存在程序目录下
def OpenNNA_Conv2d_GetWeights_From_TF(Layer_Number,Para,Weights_Index,Bias_Index,precision):
    data_t = precision#"%8.6f"
    weights_file_path = "./"+str(Layer_Number)+"_conv2d_"+str(Layer_Number)+"_weights.txt"
    Weights   = weights[Weights_Index]
    bias_file_path = "./"+str(Layer_Number)+"_conv2d_"+str(Layer_Number)+"_bias.txt"
    Bias = weights[Bias_Index]

    Conv2d_Kernel_size_row = Para[0] #卷积核大小
    Conv2d_Kernel_size_col = Para[1] #卷积核大小
    Conv2d_Kernel_channel = Para[2] #卷积核通道
    Conv2d_Kernel_number = Para[3] #卷积核数量
    Conv2d_bias_number = Para[4]  #偏置数量

    #tensorflow hwc -> chw
    loadData0 = np.swapaxes(Weights, 0, 2)
    loadData1 = np.swapaxes(loadData0, 1, 3)
    loadData2 = np.swapaxes(loadData1, 0, 1)
    txtfile0 = open(weights_file_path, 'w',encoding='UTF-8')
    for i in range(0,Conv2d_Kernel_number):
        txtfile0.write("\t{\n")
        for j in range(0,Conv2d_Kernel_channel):
            txtfile0.write("\t\t{\n")
            for k in range(0,Conv2d_Kernel_size_row):
                txtfile0.write("\t\t\t{")
                for w in range(0,Conv2d_Kernel_size_col):
                   txtfile0.write(data_t%(loadData2[i][j][k][w])+", ") 
                txtfile0.write("},\n")  
            txtfile0.write("\t\t},\n")
        txtfile0.write("\t},\n\n")
    txtfile0.close()
    #Bias
    txtfile1 = open(bias_file_path, 'w',encoding='UTF-8')
    txtfile1.write("\t{")
    for i in range(0,Conv2d_bias_number):
        txtfile1.write(data_t%(Bias[i])+", ")
    txtfile1.write("}")
    txtfile1.close()
    print("OpenNNA: Conv2d Layer:%d Weights[%d]+Bias[%d] Get Success!"%(Layer_Number,Weights_Index,Bias_Index))

## 2.从Tensorflow(.h5 File)中提取Dense的Weights

In [2]:
import numpy as np

#以下为一个提取Tensorflow Dense卷积weights+bias的函数，会以txt保存在程序目录下
def OpenNNA_Dense_GetWeights_From_TF(Layer_Number,Para,Weights_Index,Bias_Index,precision):
    data_t = precision#"%8.6f"
    weights_file_path = "./"+str(Layer_Number)+"_dense_"+str(Layer_Number)+"_weights.txt"
    Weights   = weights[Weights_Index]
    bias_file_path = "./"+str(Layer_Number)+"_dense_"+str(Layer_Number)+"_bias.txt"
    Bias = weights[Bias_Index]
    #tensorflow默认的weights存储格式为HWC，在这里需要转为OpenNNA支持的CHW
    Dense_Input_Channel = Para[0] #神经元输入channel c
    Dense_Input_Row = Para[1] #神经元输入row h
    Dense_Input_Col = Para[2] #神经元输入col w
    Dense_Units = Para[3] #神经元数量
    Dense_bias_number = Para[4]  #偏置数量

    dense1_weights = np.swapaxes(Weights,0,1)
    txtfile_1 = open(weights_file_path, 'w', encoding='UTF-8')
    txtfile_1.write("\n{")
    for i in range(0,Dense_Units):
        txtfile_1.write("\n\t{")
        for j in range(0,Dense_Input_Channel):#c
            for k in range(0,Dense_Input_Row):#h
                for l in range(0,Dense_Input_Col):#w
                    txtfile_1.write(data_t%dense1_weights[i][j+l*Dense_Input_Channel+k*Dense_Input_Channel*Dense_Input_Col]+",")
        txtfile_1.write("},\n")
    txtfile_1.write("\n}")
    txtfile_1.close()

    #提取第一层Dense的bias
    txtfile_1 = open(bias_file_path, 'w', encoding='UTF-8')
    txtfile_1.write("\n{")
    for i in range(0,Dense_bias_number):
        txtfile_1.write(data_t%Bias[i]+",")
    txtfile_1.write("}")
    txtfile_1.close()
    print("OpenNNA: Dense Layer:%d Weights[%d]+Bias[%d] Get Success!"%(Layer_Number,Weights_Index,Bias_Index))

## 3.加载训练好的模型文件(.h5 File)

In [3]:
from keras.models import load_model

model = load_model('Dense-Mnist.h5')

## 4.加载TFLite

In [1]:
import tensorflow as tf
tflite_model_name = 'Dense-Mnist-int8.tflite'
'''
Create interpreter, allocate tensors
'''
tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name)
tflite_interpreter.allocate_tensors()

'''
Check input/output details
'''
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])
print()
'''
Run prediction (optional), input_array has input's shape and dtype
'''
'''
tflite_interpreter.set_tensor(input_details[0]['index'], input_array)
tflite_interpreter.invoke()
output_array = tflite_interpreter.get_tensor(output_details[0]['index'])
'''
'''
This gives a list of dictionaries. 
'''
tensor_details = tflite_interpreter.get_tensor_details()

for dict in tensor_details:
    i = dict['index']
    tensor_name = dict['name']
    scales = dict['quantization_parameters']['scales']
    zero_points = dict['quantization_parameters']['zero_points']
    tensor = tflite_interpreter.tensor(i)()

    print(i, "|",type, "|",tensor_name, "|","Scales:",scales, "|","zero_points",zero_points, "|",tensor.shape)
    #print(tensor)
    '''
    See note below
    '''
#print("\n\n\n")
#print(tflite_interpreter.tensor(2)().shape)#tensor_details排布:List套字典

== Input details ==
name: flatten_input
shape: [ 1 28 28]
type: <class 'numpy.int8'>


== Output details ==
name: Identity
shape: [ 1 10]
type: <class 'numpy.int8'>

0 | <class 'type'> | flatten_input | Scales: [0.00392157] | zero_points [-128] | (1, 28, 28)
1 | <class 'type'> | sequential/flatten/Const | Scales: [] | zero_points [] | (2,)
2 | <class 'type'> | sequential/dense/MatMul | Scales: [0.00598583] | zero_points [0] | (128, 784)
3 | <class 'type'> | sequential/dense/BiasAdd/ReadVariableOp/resource | Scales: [2.3473825e-05] | zero_points [0] | (128,)
4 | <class 'type'> | sequential/dense_1/MatMul | Scales: [0.00680944] | zero_points [0] | (10, 128)
5 | <class 'type'> | sequential/dense_1/BiasAdd/ReadVariableOp/resource | Scales: [0.00028163] | zero_points [0] | (10,)
6 | <class 'type'> | sequential/flatten/Reshape | Scales: [0.00392157] | zero_points [-128] | (1, 784)
7 | <class 'type'> | sequential/dense/MatMul;sequential/dense/Relu;sequential/dense/BiasAdd | Scales: [0.0413582

## 5.从Tensorflow(.tflite File)中提取Dense的Weights

In [2]:
import numpy as np

#以下为一个提取Tensorflow Dense卷积weights+bias的函数，会以txt保存在程序目录下
def OpenNNA_Dense_GetWeights_From_TFLite(Layer_Number,Para,Weights_Index,Bias_Index,precision):
    tflite_interpreter = tf.lite.Interpreter(model_path='dense-mnist-tinyml.tflite')
    tflite_interpreter.allocate_tensors()
    data_t = precision#"%8.6f"
    weights_file_path = "./"+str(Layer_Number)+"_dense_"+str(Layer_Number)+"_weights.txt"
    Weights   = tflite_interpreter.tensor(Weights_Index)()
    bias_file_path = "./"+str(Layer_Number)+"_dense_"+str(Layer_Number)+"_bias.txt"
    Bias = tflite_interpreter.tensor(Bias_Index)()
    #tflite默认的weights存储格式为CHW，在这里无需要转为OpenNNA支持的CHW
    Dense_Input_Channel = Para[0] #神经元输入channel c
    Dense_Input_Row = Para[1] #神经元输入row h
    Dense_Input_Col = Para[2] #神经元输入col w
    Dense_Units = Para[3] #神经元数量
    Dense_bias_number = Para[4]  #偏置数量
    #print(Weights.shape)
    dense1_weights = Weights#np.swapaxes(Weights,0,1)
    #dense1_weights = np.swapaxes(Weights,0,1)
    print(dense1_weights.shape)
    txtfile_1 = open(weights_file_path, 'w', encoding='UTF-8')
    txtfile_1.write("\n{")
    for i in range(0,Dense_Units):
        txtfile_1.write("\n\t{")
        for j in range(0,Dense_Input_Channel):#c
            for k in range(0,Dense_Input_Row):#h
                for l in range(0,Dense_Input_Col):#w
                    #txtfile_1.write(data_t%dense1_weights[i][j+l*Dense_Input_Channel+k*Dense_Input_Channel*Dense_Input_Col]+",")
                    txtfile_1.write(data_t%dense1_weights[i][l+k*Dense_Input_Col+j*Dense_Input_Col*Dense_Input_Row]+",")
        txtfile_1.write("},\n")
    txtfile_1.write("\n}")
    txtfile_1.close()

    #提取第一层Dense的bias
    txtfile_1 = open(bias_file_path, 'w', encoding='UTF-8')
    txtfile_1.write("\n{")
    for i in range(0,Dense_bias_number):
        txtfile_1.write(data_t%Bias[i]+",")
    txtfile_1.write("}")
    txtfile_1.close()
    print("OpenNNA: Dense Layer:%d Weights[%d]+Bias[%d] Get Success!"%(Layer_Number,Weights_Index,Bias_Index))

In [3]:
OpenNNA_Dense_GetWeights_From_TFLite(1,[1,28,28,128,128],2,3,"%d")#第1层Dense
OpenNNA_Dense_GetWeights_From_TFLite(2,[1,1,128,10,10],4,5,"%d")#第2层Dense

(128, 784)
OpenNNA: Dense Layer:1 Weights[2]+Bias[3] Get Success!
(10, 128)
OpenNNA: Dense Layer:2 Weights[4]+Bias[5] Get Success!


### 将MNIST的.jpg转换为c 数组存到.txt中

In [5]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import models, layers, optimizers
import numpy as np

image_value = tf.io.read_file('./mnist_train/8_36727.jpg')

#解码为tensor
image_value = tf.io.decode_jpeg(image_value,channels = 1)


#image_value = tf.image.resize(image_value, (32,32))#改变像素值为32*32


#tensor转array
image_value = image_value.numpy()
loadData1 = np.swapaxes(image_value, 0, 2)
loadData2 = np.swapaxes(loadData1, 1, 2)

row = 28
col = 28

txtfile = open(r'image_8_36727_int8.txt', 'w',encoding='UTF-8')
for i in range(0,1):
    txtfile.write("\t{\n")
    for j in range(0,row):
        txtfile.write("\t\t{")
        for k in range(0,col):
            #txtfile.write(str(loadData2[i][j][k])+", ") 
            txtfile.write("%d"%(loadData2[i][j][k]-128)+", ") 
        txtfile.write("},\n")
    txtfile.write("\t},\n\n")
txtfile.close()